```{contents}
```
## **Loop in LangGraph**

A **loop** in LangGraph is a **control-flow construct** that enables **repeated execution of graph nodes** until a termination condition is satisfied.
Loops are fundamental for building **iterative reasoning, tool-use cycles, self-correction, planning, and autonomous agents**.

---

### **1. Why Loops Matter**

Most intelligent tasks require iteration:

| Task Type    | Why Loop                       |
| ------------ | ------------------------------ |
| Reasoning    | Gradual refinement of solution |
| Tool Use     | Feedback-driven improvement    |
| Planning     | Plans evolve                   |
| Verification | Detect and fix errors          |
| Autonomy     | Continue until goal achieved   |

Without loops, workflows are **static pipelines**.
With loops, LangGraph supports **adaptive, self-directed systems**.

---

### **2. How LangGraph Implements Loops**

LangGraph uses **graph cycles** instead of `while` loops.
Control is governed by **state transitions + conditional edges**.

**Core elements:**

1. **Loop Body** — nodes that repeat
2. **State** — shared memory updated each iteration
3. **Router** — decides continue vs terminate
4. **Exit Condition** — encoded in state

---

### **3. Conceptual Execution Model**

```
Entry
  ↓
[ Step A ] → [ Step B ] → [ Check ]
    ↑                       ↓
    └────────── Loop ───────┘
```

---

### **4. Minimal Loop Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    counter: int

def increment(state):
    return {"counter": state["counter"] + 1}

def check(state):
    return {"done": state["counter"] >= 5}

builder = StateGraph(State)

builder.add_node("inc", increment)
builder.add_node("check", check)

builder.set_entry_point("inc")
builder.add_edge("inc", "check")

builder.add_conditional_edges(
    "check",
    lambda s: END if s["done"] else "inc",
    {"inc": "inc", END: END}
)

graph = builder.compile()
print(graph.invoke({"counter": 0}))
```

---

### **5. Real-World Loop Patterns**

| Pattern           | Purpose                     |
| ----------------- | --------------------------- |
| ReAct Loop        | Reason → Act → Observe      |
| Reflection Loop   | Draft → Critique → Revise   |
| Planner–Executor  | Plan → Execute → Replan     |
| Verification Loop | Generate → Test → Fix       |
| Autonomous Agent  | Goal → Plan → Act → Monitor |

---

### **6. Loop Variants**

| Variant          | Behavior                |
| ---------------- | ----------------------- |
| Finite Loop      | Stops after condition   |
| Bounded Loop     | Max iterations enforced |
| Infinite Loop    | No termination (unsafe) |
| Conditional Loop | Controlled by state     |
| Nested Loop      | Loop inside subgraph    |
| Parallel Loop    | Concurrent iterations   |

---

### **7. Safety & Production Controls**

| Control           | Purpose                |
| ----------------- | ---------------------- |
| `recursion_limit` | Prevent infinite loops |
| Timeouts          | Stop stalled runs      |
| Checkpointing     | Recover mid-loop       |
| Human interrupt   | Manual stop            |
| State logging     | Full auditability      |

```python
graph.invoke(input, config={"recursion_limit": 20})
```

---

### **8. Design Guidelines**

* Always encode **exit condition** in state
* Enforce **max iteration limits**
* Log each loop iteration
* Validate state changes
* Use checkpoints for long loops

---

### **9. Mental Model**

A LangGraph loop behaves like a **closed feedback control system**:

> **State → Decision → Action → Feedback → State**

This is the foundation of **agent intelligence and autonomy**.



### Demonstration

In [1]:
from typing import TypedDict

class State(TypedDict):
    task: str
    draft: str
    score: int

def reason(state):
    draft = f"{state['task']} — attempt {state['score']+1}"
    return {"draft": draft}

def critique(state):
    score = state["score"] + 1
    return {"score": score}

def should_continue(state):
    if state["score"] >= 3:
        return "final"
    return "reason"

def final(state):
    return {"draft": f"FINAL ANSWER: {state['draft']}"}




In [2]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("reason", reason)
builder.add_node("critique", critique)
builder.add_node("final", final)

builder.set_entry_point("reason")
builder.add_edge("reason", "critique")

builder.add_conditional_edges(
    "critique",
    should_continue,
    {
        "reason": "reason",
        "final": "final"
    }
)

builder.add_edge("final", END)

graph = builder.compile()


In [3]:
result = graph.invoke({
    "task": "Write a concise definition of overfitting",
    "draft": "",
    "score": 0
}, config={"recursion_limit": 10})

print(result["draft"])

FINAL ANSWER: Write a concise definition of overfitting — attempt 3
